In [577]:
import pandas as pd

In [578]:
df_global = pd.read_csv("/Users/davidemolinelli/Downloads/simulations/4/flash_mob/global-indicators.csv")[["timestamp","rides_canceled","rides_not_accomplished","rides_completed","rides_accepted","total_rides"]]
df_specific = pd.read_csv("/Users/davidemolinelli/Downloads/simulations/4/flash_mob/specific-indicators.csv")[["timestamp","expected_total_time","total_time","expected_ride_price","ride_price"]]

In [579]:
df_global.head()

,timestamp,rides_canceled,rides_not_accomplished,rides_completed,rides_accepted,total_rides
0,1.0,0,0,0,0,0
1,2.0,0,0,0,0,2
2,3.0,0,0,0,0,2
3,4.0,0,0,0,0,2
4,5.0,0,0,0,0,2


In [580]:
df_specific[113:117]

,timestamp,expected_total_time,total_time,expected_ride_price,ride_price
113,1900.0,247.90,728.0,21.52,68.88
114,1906.0,210.11,634.0,32.97,85.62
115,1953.0,200.77,254.0,26.39,7.19
116,2004.0,255.09,461.0,31.51,51.32


In [581]:
df_global['unserved_requests_100'] = 0
df_global['unserved_requests_200'] = 0
df_global['unserved_requests_500'] = 0
#df_global['unserved_requests_100_min'] = 0
#df_global['unserved_requests_200_min'] = 0
#df_global['unserved_requests_500_min'] = 0
#df_global['unserved_requests_100_max'] = 0
#df_global['unserved_requests_200_max'] = 0
#df_global['unserved_requests_500_max'] = 0
df_global['overhead_100'] = 0
df_global['overhead_200'] = 0
df_global['overhead_500'] = 0
#df_global['overhead_100_min'] = 0
#df_global['overhead_200_min'] = 0
#df_global['overhead_500_min'] = 0
#df_global['overhead_100_max'] = 0
#df_global['overhead_200_max'] = 0
#df_global['overhead_500_max'] = 0
df_global['price_fluctuation_100'] = 0
df_global['price_fluctuation_200'] = 0
df_global['price_fluctuation_500'] = 0

In [582]:
for i in range(100,5000):
    df_end_window = df_global.iloc[i-1,:]
    df_start_window = df_global.iloc[i-100,:]
    df_window_num = (df_end_window["rides_canceled"] - df_start_window["rides_canceled"]) + (df_end_window["rides_not_accomplished"] - df_start_window["rides_not_accomplished"])
    df_window_den = (df_end_window["total_rides"] + df_end_window["rides_accepted"]) - (df_start_window["total_rides"] + df_start_window["rides_accepted"])
    df_global.at[i-1,"unserved_requests_100"] = df_window_num / df_window_den if not df_window_den == 0 else 0

for i in range(200,5000):
    df_end_window = df_global.iloc[i-1,:]
    df_start_window = df_global.iloc[i-200,:]
    df_window_num = (df_end_window["rides_canceled"] - df_start_window["rides_canceled"]) + (df_end_window["rides_not_accomplished"] - df_start_window["rides_not_accomplished"])
    df_window_den = (df_end_window["total_rides"] + df_end_window["rides_accepted"]) - (df_start_window["total_rides"] + df_start_window["rides_accepted"])
    df_global.at[i-1,"unserved_requests_200"] = df_window_num / df_window_den if not df_window_den == 0 else 0

for i in range(500,5000):
    df_end_window = df_global.iloc[i-1,:]
    df_start_window = df_global.iloc[i-500,:]
    df_window_num = (df_end_window["rides_canceled"] - df_start_window["rides_canceled"]) + (df_end_window["rides_not_accomplished"] - df_start_window["rides_not_accomplished"])
    df_window_den = (df_end_window["total_rides"] + df_end_window["rides_accepted"]) - (df_start_window["total_rides"] + df_start_window["rides_accepted"])
    if df_window_num < 0:
        print(i)
        print(df_window_num)
    if df_window_den < 0:
        print(i)
        print(df_end_window)
    df_global.at[i-1,"unserved_requests_500"] = df_window_num / df_window_den if not df_window_den == 0 else 0

In [583]:
df_specific["overhead"] = df_specific.apply(lambda x: abs(x["total_time"]-x["expected_total_time"])/(x["expected_total_time"]), axis=1)
df_specific["price_fluctuation"] =  df_specific.apply(lambda x: abs(x["ride_price"]-x["expected_ride_price"])/(x["expected_ride_price"]), axis=1)
df_specific.head()

,timestamp,expected_total_time,total_time,expected_ride_price,ride_price,overhead,price_fluctuation
0,332.0,105.12,293.0,16.27,40.09,1.787291,1.464044
1,458.0,138.51,365.0,13.93,25.60,1.635189,0.837760
2,460.0,122.55,441.0,14.17,30.67,2.598531,1.164432
3,463.0,232.57,438.0,23.15,44.47,0.883304,0.920950
4,470.0,217.40,414.0,27.31,48.09,0.904324,0.760893


In [584]:
for i in range(100,5000):
    df_filtered = df_specific[(float(i-100) <= df_specific["timestamp"]) & (df_specific["timestamp"] <= float(i))].reset_index()
    df_global.at[i-1,"overhead_100"] = df_filtered["overhead"].mean()
    df_global.at[i-1,'price_fluctuation_100'] = df_filtered["price_fluctuation"].mean()
for i in range(200,5000):
    df_filtered = df_specific[(float(i-200) <= df_specific["timestamp"]) & (df_specific["timestamp"] <= float(i))].reset_index()
    df_global.at[i-1,"overhead_200"] = df_filtered["overhead"].mean()
    df_global.at[i-1,'price_fluctuation_200'] = df_filtered["price_fluctuation"].mean()
for i in range(400,5000):
    df_filtered = df_specific[(float(i-500) <= df_specific["timestamp"]) & (df_specific["timestamp"] <= float(i))].reset_index()
    df_global.at[i-1,"overhead_500"] = df_filtered["overhead"].mean()
    df_global.at[i-1,'price_fluctuation_500'] = df_filtered["price_fluctuation"].mean()

In [585]:
df_global = df_global.fillna(0)

In [586]:
print(df_global.iloc[200,:])

timestamp                 201.000000
rides_canceled              0.000000
rides_not_accomplished      5.000000
rides_completed             0.000000
rides_accepted             20.000000
total_rides                 6.000000
unserved_requests_100       0.444444
unserved_requests_200       0.208333
unserved_requests_500       0.000000
overhead_100                0.000000
overhead_200                0.000000
overhead_500                0.000000
price_fluctuation_100       0.000000
price_fluctuation_200       0.000000
price_fluctuation_500       0.000000
Name: 200, dtype: float64


In [587]:
df_global.to_csv("../analysis/v4/underground.csv", index=False)

In [588]:
unserved_100_mean = df_global["unserved_requests_100"].mean()
unserved_200_mean = df_global["unserved_requests_200"].mean()
unserved_500_mean = df_global["unserved_requests_500"].mean()
unserved_100_min = df_global["unserved_requests_100"].mean()
unserved_200_min = df_global["unserved_requests_200"].mean()
unserved_500_min = df_global["unserved_requests_500"].mean()
unserved_100_min = df_global["unserved_requests_100"].mean()
unserved_200_min = df_global["unserved_requests_200"].mean()
unserved_500_min = df_global["unserved_requests_500"].mean()

In [105]:
df_specific["overhead"] = df_specific.apply(lambda x: (x["total_time"]-x["expected_total_time"])/(x["expected_total_time"]), axis=1)
df_specific.head()

,timestamp,expected_total_time,total_time,expected_ride_price,ride_price,overhead,price_fluctuation
0,332.0,165.50,325.0,19.24,30.69,0.963746,0.963746
1,366.0,171.42,272.0,19.64,30.49,0.586746,0.586746
2,397.0,110.30,222.0,16.22,28.44,1.012693,1.012693
3,407.0,183.67,387.0,18.96,37.26,1.107040,1.107040
4,416.0,110.27,333.0,11.73,32.75,2.019860,2.019860


In [64]:
df_global.head()

,timestamp,rides_canceled,rides_not_accomplished,rides_completed,rides_accepted,total_rides
0,1.0,0,0,0,0,0
1,2.0,0,0,0,0,2
2,3.0,0,0,0,0,2
3,4.0,0,0,0,0,2
4,5.0,0,0,0,0,2


In [ ]:
df_window = df_global.iloc[0:500,:]
df_window["unserved_numerator"] = df_window[]
df_window.head()

In [ ]:
df_499 = df_global.iloc[499,:]
df_499

In [ ]:
df_num = df_global.iloc[499,:]["rides_canceled"] - df_global.iloc[0,:]["rides_canceled"] +  df_global.iloc[499,:]["rides_not_accomplished"] - df_global.iloc[0,:]["rides_not_accomplished"]
df_num